In [1]:
import numpy as np
import cv2
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

In [2]:
def detect_lines(img):
    lines_to_return = []
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges_img = cv2.Canny(gray_img, 50, 150, apertureSize=3)
    
    min_line_length = 100
    
    lines = cv2.HoughLinesP(image=edges_img, rho=1, theta=np.pi/180, threshold=10, lines=np.array([]),
                            minLineLength=min_line_length, maxLineGap=20)
    
    for line in lines:
        x1 = line[0][0]
        y1 = line[0][1]
        x2 = line[0][2]
        y2 = line[0][3]
        lines_to_return.append([x1, y1, x2, y2])
        
    return lines_to_return

In [3]:
def process_video(video_path):
    minline = {}
    maxline = {}
    
    number_of_hits = 0
    hits_left = 0
    hits_rigth = 0
    last_hit_left = 0
    last_hit_rigth = 0
    frame_num = 0
    cap = cv2.VideoCapture(video_path)
    cap.set(1, frame_num)
    
    while True:
        frame_num += 1
        ret_val, frame = cap.read()
        if not ret_val:
            return hits_left + hits_rigth
        
        if frame_num == 1: 
            lines = detect_lines(frame)
            minline = [1000,1000,1000,1000]
            maxline = [-1, -1, -1, -1]
            for line in lines:
                if line[2] == line[0]:

                    if(line[0] > maxline[0]):
                        maxline = line
                    if(line[0] < minline[0]):
                        minline = line

        else:
            img = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) 
            
            ret, img_bin = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY)
            
            contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            img = frame.copy()

            for contour in contours:
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                if radius > 3.6 and radius < 5:
                    if abs(minline[0] - x) < 18.5:
                        if last_hit_left == frame_num - 1:
                            continue
                        last_hit_left = frame_num
                        hits_left += 1
                    if abs(maxline[0] - x) < 18.5:
                        if last_hit_rigth == frame_num - 1:
                            continue
                        last_hit_rigth = frame_num
                        hits_rigth += 1

In [4]:
res = []
correct_res = [7, 18, 21, 18, 10, 32, 13, 15, 14, 24]
res.append(process_video("data/video1.mp4"))
res.append(process_video("data/video2.mp4"))
res.append(process_video("data/video3.mp4"))
res.append(process_video("data/video4.mp4"))
res.append(process_video("data/video5.mp4"))
res.append(process_video("data/video6.mp4"))
res.append(process_video("data/video7.mp4"))
res.append(process_video("data/video8.mp4"))
res.append(process_video("data/video9.mp4"))
res.append(process_video("data/video10.mp4"))

print(res)
print(correct_res)
print(mean_absolute_error(correct_res, res))

with open('my-res.txt', 'w') as f:
    f.write('file,count')
    f.write('\n')
    i = 1
    for number in res:
        f.write('video' + str(i) + '.mp4,' + str(number))
        f.write('\n')
        i += 1

[7, 18, 21, 18, 10, 31, 13, 15, 14, 24]
[7, 18, 21, 18, 10, 32, 13, 15, 14, 24]
0.1
